In [5]:
%pip install -qU google-api-python-client


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [7]:
import os

In [9]:
from google.cloud import aiplatform

# This is used by the LiteLLM for Vertex AI models including Gemini.
# The LiteLLM wrapper for Gemini is used by the TruLens evaluation provider.
# aiplatform.init(project="wiki3-389323", location="us-west1")
# Dunno why PUP-GPT project is selected and I don't think this affects how/why it is the one being used.
aiplatform.init(project="pup-gpt", location="us-west1")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True, temperature=0)
response = chat_model.invoke("Hello")
response

AIMessage(content='Hello! How can I assist you today?')

In [3]:
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnablePassthrough

from research_assistant.search.web import get_search_queries
from research_assistant.search.web import chain as search_chain
from research_assistant.writer import chain as writer_chain

chain_notypes = (
    RunnablePassthrough().assign(research_summary=search_chain) | writer_chain
)


class InputType(BaseModel):
    question: str


chain = chain_notypes.with_types(input_type=InputType)


In [26]:
get_search_queries.invoke(input={'question':'where is the best place to live in the world?'})

['Best places to live in the world 2023',
 'Factors to consider when choosing a place to live',
 'Pros and cons of living in different countries']

In [12]:
# Can't use this query because it fails with a medium safety risk for harassment.
# Fix (i.e. ability to change default safety settings) is in progress: https://github.com/langchain-ai/langchain/pull/15107
response = search_chain.invoke(input={'question':"What is the meaning of life?"})
print(response)

BlockedPromptException: block_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}


In [6]:
response = search_chain.invoke(input={'question':"What is the best Linked Open Data (LOD) store to use with Python and LangChain?"})
print(response)

In [10]:
response = search_chain.invoke(input={'question':"I need a wiki solution that does provenance using Linked Open Data (LOD) in Python."})
print(response)

Source Url: https://en.wikipedia.org/wiki/Open_energy_system_databases
Summary: The provided text does not contain any information about Python wiki solution provenance Linked Open Data (LOD). Therefore, I cannot answer your question.

The text discusses open energy system databases, their benefits, challenges, and various projects that collect, collate, document, and republish energy system datasets. It also mentions the importance of open data, data models, licensing, and data standards in the context of energy system modeling.
Source Url: https://www.mdpi.com/2673-6470/2/3/19
Summary: The provided text does not contain any information about Python wiki solution provenance Linked Open Data (LOD). Therefore, I cannot answer your question.
Source Url: https://en.wikipedia.org/wiki/Provenance
Summary: The provided text does not contain any information about Python wiki solution provenance Linked Open Data (LOD). Therefore, I cannot answer your question.

Source Url: https://en.wikipedia

In [ ]:
response = chain.invoke(input={'question':'where is the best place to live in the world?'})